In [1]:
import pandas as pd
import os
import sys
import numpy as np

from datasketch import MinHash, MinHashLSH

In [2]:
align_table_path = "/scratch/indikar_root/indikar1/shared_data/single_cell/align_table/o1b12.GRCm39.align_table.parquet"

df = pd.read_parquet(align_table_path)
print(f"{df.shape=}")
df.head()

df.shape=(335037, 15)


,read_name,align_id,read_start,read_end,length_on_read,chrom,ref_start,ref_end,fragment_id,fragment_start,fragment_end,fragment_length,monomer_duplicate,is_mapped,mapping_quality
0,0000f5e6-aa81-4c68-aaeb-a1eedd07004a,285546,0,25,25,NaN,-1,NaN,NaN,NaN,NaN,NaN,False,False,0
1,0000f5e6-aa81-4c68-aaeb-a1eedd07004a,285547,25,195,170,8,5387429,5387606.0,5451022.0,5387435.0,5387612.0,177.0,False,True,60
2,0000f5e6-aa81-4c68-aaeb-a1eedd07004a,285548,195,218,23,NaN,-1,NaN,NaN,NaN,NaN,NaN,False,False,0
3,0000f5e6-aa81-4c68-aaeb-a1eedd07004a,285549,218,241,23,NaN,-1,NaN,NaN,NaN,NaN,NaN,False,False,0
4,0000f5e6-aa81-4c68-aaeb-a1eedd07004a,285550,241,264,23,NaN,-1,NaN,NaN,NaN,NaN,NaN,False,False,0


In [3]:
def preliminary_filters(df):
    """A set of filters to remove unmapped monomers"""
    df = df[df['is_mapped']]
    return df

df = preliminary_filters(df)
print(f"{df.shape=}")
df.head()

df.shape=(218771, 15)


,read_name,align_id,read_start,read_end,length_on_read,chrom,ref_start,ref_end,fragment_id,fragment_start,fragment_end,fragment_length,monomer_duplicate,is_mapped,mapping_quality
1,0000f5e6-aa81-4c68-aaeb-a1eedd07004a,285547,25,195,170,8,5387429,5387606.0,5451022.0,5387435.0,5387612.0,177.0,False,True,60
5,0000f5e6-aa81-4c68-aaeb-a1eedd07004a,285551,264,342,78,4,76192837,76192911.0,2893754.0,76192843.0,76192917.0,74.0,False,True,3
8,0000f5e6-aa81-4c68-aaeb-a1eedd07004a,285554,457,690,233,6,100212019,100212254.0,4489906.0,100212021.0,100212261.0,240.0,False,True,60
11,00016c35-6d3c-4aa8-9898-2ecd944bd32d,274157,81,146,65,1,56932849,56932914.0,256172.0,56932855.0,56932972.0,117.0,False,True,36
12,00016c35-6d3c-4aa8-9898-2ecd944bd32d,274158,146,347,201,11,44091729,44091933.0,7484648.0,44091731.0,44091918.0,187.0,False,True,60


In [4]:
def concatamer2list(group):
    """
    Efficiently joins fragment ID codes into a semicolon-separated string, excluding -1.
    
    Parameters:
        group (pandas.Series): Series containing fragment ID codes.
    
    Returns:
        str: Semicolon-separated string of fragment ID codes.
    """
    return ";".join(sorted(set(str(x) for x in group.dropna())))


def flag_exact_duplicates(df):
    # Assign categorical codes to fragment IDs
    df['code'] = df['fragment_id'].astype('category').cat.codes

    # Aggregate information efficiently
    df = (
        df.groupby('read_name')
        .agg(mapq=('mapping_quality', 'mean'), 
             fragments=('code', concatamer2list), 
             order=('fragment_id', 'nunique'))
        .reset_index(drop=False)
    )

    df['exact_duplicate_group'] = df.groupby('fragments').ngroup() # add read_group id
    df = df[['read_name', 'mapq',  'order', 'exact_duplicate_group', 'fragments',]]
    return df

pdf = flag_exact_duplicates(df)
print(f"{pdf.shape=}")
pdf.head()

pdf.shape=(44742, 5)


,read_name,mapq,order,exact_duplicate_group,fragments
0,0000f5e6-aa81-4c68-aaeb-a1eedd07004a,41.000000,3,2700,1383;2153;2614
1,00016c35-6d3c-4aa8-9898-2ecd944bd32d,21.857143,7,1413,123;1579;1580;1581;3816;5552;690
2,0001ce59-8520-4d81-a780-259bd7d932c4,31.000000,7,9542,3810;3811;3812;5151;5152;6101;6300
3,000381df-b128-4e36-8667-f26d0c297632,60.000000,2,10581,5340;637
4,000399b0-c502-4fac-8f86-f481550f751a,60.000000,6,10363,4960;4961;4962;4963;4964;4965


In [5]:
def find_similar_entries_minhash(arr, threshold=0.5, num_perm=128):
    """Finds similar entries in an array using MinHash for Jaccard similarity approximation.

    Args:
        arr (numpy.ndarray): The input array of strings representing sets.
        threshold (float): The minimum Jaccard similarity for two sets to be considered similar.
        num_perm (int): The number of permutations used for MinHash.

    Returns:
        list: A list of lists, where each inner list contains similar entries.
    """
    
    minhashes = {}
    lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)

    for i, entry in enumerate(arr):
        m = MinHash(num_perm=num_perm)
        for val in map(int, entry.split(';')):
            m.update(str(val).encode('utf8'))  # Correct encoding for integers
        lsh.insert(str(i), m)
        minhashes[i] = m

    similar_groups = []
    processed = set()
    for i in range(len(arr)):
        if i in processed:
            continue
        similar = [arr[i]]
        result = lsh.query(minhashes[i])  
        for key in result:
            j = int(key)
            if j != i and j not in processed:  # Avoid self and duplicates
                similar.append(arr[j])
                processed.add(j)  # Mark as processed
        if len(similar) > 1:
            similar_groups.append(similar)
    return similar_groups


near_read_groups = find_similar_entries_minhash(pdf['fragments'].values, threshold=0.6)
len(near_read_groups)

2041

In [6]:
def annotate_near_duplicates(df, near_read_groups):
    """A function to annotate near duplicates """
    
    df['near_duplicate_group'] = -1
    
    for i, group in enumerate(near_read_groups):
        codelist = list(set(group))
        
        df['near_duplicate_group'] = np.where(df['fragments'].isin(group), i, df['near_duplicate_group'])
    
    # add a unique id for each read that didn't fall into a 
    # duplicate group

    # Find the maximum existing ID
    max_id = df['near_duplicate_group'].max()

    # Create a mask for rows with unknown IDs
    unknown_mask = df['near_duplicate_group'] == -1

    # Generate sequential IDs starting from the max + 1
    new_ids = range(max_id + 1, max_id + 1 + unknown_mask.sum())

    # Replace unknown IDs with the new sequential IDs
    df.loc[unknown_mask, 'near_duplicate_group'] = new_ids
    
    return df
        
        
tdf = annotate_near_duplicates(pdf, near_read_groups)
tdf.head()

,read_name,mapq,order,exact_duplicate_group,fragments,near_duplicate_group
0,0000f5e6-aa81-4c68-aaeb-a1eedd07004a,41.000000,3,2700,1383;2153;2614,0
1,00016c35-6d3c-4aa8-9898-2ecd944bd32d,21.857143,7,1413,123;1579;1580;1581;3816;5552;690,1
2,0001ce59-8520-4d81-a780-259bd7d932c4,31.000000,7,9542,3810;3811;3812;5151;5152;6101;6300,2
3,000381df-b128-4e36-8667-f26d0c297632,60.000000,2,10581,5340;637,3
4,000399b0-c502-4fac-8f86-f481550f751a,60.000000,6,10363,4960;4961;4962;4963;4964;4965,4


In [7]:
def annotate_duplicates(df):
    """A function to annotate duplicate reads """
    df['unique'] = df.groupby('near_duplicate_group')['mapq'].transform(pd.Series.rank, method='first', ascending=False) == 1
    return df


tdf = annotate_duplicates(tdf)
tdf.head()

,read_name,mapq,order,exact_duplicate_group,fragments,near_duplicate_group,unique
0,0000f5e6-aa81-4c68-aaeb-a1eedd07004a,41.000000,3,2700,1383;2153;2614,0,False
1,00016c35-6d3c-4aa8-9898-2ecd944bd32d,21.857143,7,1413,123;1579;1580;1581;3816;5552;690,1,False
2,0001ce59-8520-4d81-a780-259bd7d932c4,31.000000,7,9542,3810;3811;3812;5151;5152;6101;6300,2,False
3,000381df-b128-4e36-8667-f26d0c297632,60.000000,2,10581,5340;637,3,True
4,000399b0-c502-4fac-8f86-f481550f751a,60.000000,6,10363,4960;4961;4962;4963;4964;4965,4,True


In [8]:
tdf['unique'].value_counts()

unique
False    40297
True      4445
Name: count, dtype: int64

In [9]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
tdf[tdf['near_duplicate_group'] == 8]

In [ ]:
break

In [ ]:
near_read_groups[0]

In [ ]:
break

In [ ]:
def find_similar_entries_jaccard(arr, threshold=0.5):
    """Finds similar entries in an array based on Jaccard similarity.

    Args:
        arr (numpy.ndarray): The input array of strings representing sets.
        threshold (float): The minimum Jaccard similarity for two sets to be considered similar.

    Returns:
        list: A list of lists, where each inner list contains similar entries.
    """
    sets = [set(map(int, entry.split(';'))) for entry in arr]
    similar_groups = []
    for i, set1 in enumerate(sets):
        similar = [arr[i]]  # Start with the current entry
        for j, set2 in enumerate(sets[i+1:], start=i+1):
            jaccard_sim = len(set1 & set2) / len(set1 | set2)  # Calculate Jaccard similarity
            if jaccard_sim >= threshold:
                similar.append(arr[j])
        if len(similar) > 1:  # Add the group if it has more than one entry
            similar_groups.append(similar)
    return similar_groups


find_similar_entries_jaccard(pdf['codelist'].values, threshold=0.8)

In [ ]:
break

In [ ]:
def flag_near_duplicates(df, id_column='fragment_id', threshold=0.6):
    """
    Group near duplicates in a DataFrame based on feature similarity.

    Jaccard similarity coefficient, defined as the size of the
    intersection divided by the size of the union of two label sets

    Parameters:
        df (pd.DataFrame): DataFrame with rows representing samples and columns representing binary features.
        threshold (float): Threshold for similarity (distance) to consider rows as near duplicates. Defaults to 0.5.
            lower numbers are more stringent, high numbers are more liberal when flagging reads as near duplicates

    Returns:
        tuple: Tuple containing uniqueness map and read group map.
    """
    # Filter based on combined conditions
    mask = (df['exact_unique'] & 
          (df.groupby('read_name')[id_column].transform('nunique') > 2) & 
          (df.groupby(id_column)['read_name'].transform('nunique') > 2))
    df = df[mask].reset_index(drop=True)


    # pivot the dataframe so that each row
    # represent a single read and each column represents
    # each mapped monomer
    df['val'] = 1
    df = pd.pivot_table(df, 
                        index='read_name',
                        columns=id_column,
                        values='val',
                        fill_value=0).astype(bool)
     
    # sort by "cardinality" so that we can bias
    # selection of higher-order, i.e., more 'complete' reads 
    df = df.sort_index(level=id_column, 
                       ascending=False)

    # store read names
    read_names = df.index
    
    # Convert df to sparse CSR matrix
    df = csc_matrix(df)

    # Calculate pairwise Jaccard distance between rows using sparse operations
    distances = pdist(df.toarray(), metric='jaccard')

    # if no eligible reads, return
    if distances.size == 0:
        return {}, {}

    # Perform hierarchical clustering to identify 
    # near duplicates
    clusters = fcluster(linkage(distances, 
                                method='complete', 
                                metric='jaccard'), 
                        t=threshold, 
                        criterion='distance')

   # Create final DataFrame efficiently
    dupes = pd.DataFrame({
        'read_name': read_names,
        'read_group': clusters,
        'order': read_names.map(read_names.value_counts()),  # Get 'order' efficiently
    })

    # flag the largest concatemer as the one kept
    # TODO: this criteria biases concatemer selection to potentially over-digested
    # sets of monomers - potential fix to weight selection of
    # 'best' concatemer of duplicate group by mappability and monomer
    # total bases covered on read/reference
    dupes['exact_unique'] = dupes.groupby('read_group')['order'].transform(pd.Series.rank, method='first', ascending=False) == 1

    uniqueness_map = dict(zip(dupes['read_name'].values, dupes['exact_unique'].values))
    read_group_map = dict(zip(dupes['read_name'].values, dupes['read_group'].values))
    return uniqueness_map, read_group_map